In [ ]:
อยากจะเพิ่ม genres_bias เข้าไป แต่ไม่รู้จะใส่ค่าอะไรลงไปในคอลัมน์ not_all_ratings["genres"]
ต้องมีshape (9742, 9742) ไหม หรือจะทำอะไรกับ flat_list ของ genres ที่ userinput ให้มา
not_all_ratings.loc[:, "genres"] = movies_not_watched["genres"]. อะไรสักอย่าง 

liked_genres = all_movies.loc[all_movies['movieId'].isin(watched_movie_id_list), 'genres'].to_list()
#['Action|Sci-Fi', 'Mystery|Sci-Fi|Thriller', 'Sci-Fi|IMAX']

splitted = [sub.split("|") for sub in liked_genres]
flat_genres_list = [item for sublist in splitted for item in sublist]
#['Action', 'Sci-Fi', 'Mystery', 'Sci-Fi', 'Thriller', 'Sci-Fi', 'IMAX']
shape คือ (7,7)
คือรู้แล้วว่าขาเข้าต้องเป็น 9742 แต่ขาออกมาที่เป็น movie_bias แล้วรูปร่างหน้าตาเป็นอย่างไร
จึงอยากจะเรียก class RecommenderNet() มาดูสักอัน
โดยต้อง instantiate class ก่อน
recommender = RecommenderNet()

class RecommenderNet(keras.Model):
        def __init__(self, num_users, num_movies, embedding_size, **kwargs):
            super(RecommenderNet, self).__init__(**kwargs)
            
แล้ว forward pass ด้วยการเรียก method call

        def call(self, inputs):

แต่ไม่เข้าใจ inputs มาจากไหน แล้วทำไมเรียก recommender.call() ไม่ได้

นี่คือเรา subclass มาใช่มั้ย
 By subclassing the Model class: in that case, you should define your layers in __init__ 
and you should implement the model's forward pass in call.

In [1]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import create_engine
import os
import pandas as pd

app = Flask(__name__)

engine = create_engine('sqlite:///recommender2.db', echo=False)

for f in os.listdir('data/movies/ml-latest-small'):
    if f[-4:] == '.csv':
        data = pd.read_csv(f'data/movies/ml-latest-small/{f}')
        data.to_sql(f[:-4], engine)
        print(f[0:-4])

links
ratings
movies
tags


In [2]:
watched_movie_id_list = ['70286', '109487', '589']

In [19]:
import numpy as np

movie_id_unique = 'SELECT * FROM movies'
all_movies = pd.read_sql(movie_id_unique, engine)

#remove the input movies from all_movies
movies_not_watched = all_movies[~all_movies['movieId'].isin(watched_movie_id_list)]
movies_not_watched.loc[:,'fake_id'] = np.ones(len(movies_not_watched), dtype =int)

#get all_ratings from sqlite
query = 'SELECT "userId", ratings."movieId", movies.title, rating FROM ratings JOIN movies ON ratings."movieId" = movies."movieId";'
all_ratings = pd.read_sql(query, engine)

#remove the watched movies from all_ratings
not_all_ratings = all_ratings[~all_ratings['movieId'].isin(watched_movie_id_list)]

#remove movieId and ratings of the watched movies???
movieindex = not_all_ratings['movieId'].unique().tolist()
dl_movie2movie_encoded = {x: i for i, x in enumerate(movieindex)}
dl_movie_encoded2movie = {i: x for i, x in enumerate(movieindex)}

not_all_ratings.loc[:,"movie"] = not_all_ratings["movieId"].map(dl_movie2movie_encoded)
not_all_ratings.loc[:,"rating"] = not_all_ratings["rating"].values.astype(np.float32)

#map userId from ?? to not_all_ratings ?? what do i do with the users??

not_all_user_ids = not_all_ratings["userId"].unique().tolist()
dl_user2user_encoded = {x: i for i, x in enumerate(not_all_user_ids)}
dl_userencoded2user = {i: x for i, x in enumerate(not_all_user_ids)}

not_all_ratings.loc[:,"user"] = not_all_ratings["userId"].map(dl_user2user_encoded)

min_rating = min(not_all_ratings["rating"])
max_rating = max(not_all_ratings["rating"])
num_users = len(dl_user2user_encoded)
num_movies = len(dl_movie_encoded2movie)
print(
        "Number of users: {}, Number of Movies: {}, Min rating: {}, Max rating: {}".format(
            num_users, num_movies, min_rating, max_rating
        )
    )

#define training data
df = not_all_ratings.sample(frac=1, random_state=42)
x = not_all_ratings[["user", "movie"]].values

    # Normalize the targets between 0 and 1. Makes it easy to train.
y = not_all_ratings["rating"].apply(lambda x: (x - min_rating) / (max_rating - min_rating)).values
    # Assuming training on 90% of the data and validating on 10%.
train_indices = int(0.9 * df.shape[0])
x_train, x_val, y_train, y_val = (
        x[:train_indices],
        x[train_indices:],
        y[:train_indices],
        y[train_indices:],
    )

#inference
user_movie_array = movies_not_watched[['fake_id','index']]
max_movie_index = user_movie_array['index'].max()
that = user_movie_array.to_numpy()

EMBEDDING_SIZE = 50

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input

class RecommenderNet(keras.Model):
        def __init__(self, num_users, num_movies, embedding_size, **kwargs):
            super(RecommenderNet, self).__init__(**kwargs)
            self.num_users = num_users
            self.num_movies = num_movies
            self.embedding_size = embedding_size
            self.user_embedding = layers.Embedding(
                num_users,
                embedding_size,
                embeddings_initializer="he_normal",
                embeddings_regularizer=keras.regularizers.l2(1e-6),
            )
            self.user_bias = layers.Embedding(num_users, 1)
            self.movie_embedding = layers.Embedding(
            #change this line from num_movies to max_movie_index+1
                max_movie_index+1,
                embedding_size,
                embeddings_initializer="he_normal",
                embeddings_regularizer=keras.regularizers.l2(1e-6),
            )
            #and this line (input_dim)
            self.movie_bias = layers.Embedding(max_movie_index+1, 1)

        def call(self, inputs):
            user_vector = self.user_embedding(inputs[:, 0])
            user_bias = self.user_bias(inputs[:, 0])
            movie_vector = self.movie_embedding(inputs[:, 1])
            movie_bias = self.movie_bias(inputs[:, 1])
            print(movie_bias)
            dot_user_movie = tf.tensordot(user_vector, movie_vector, 2)
            print(dot_user_movie)
            # Add all the components (including bias)
            x = dot_user_movie + user_bias + movie_bias
            # The sigmoid activation forces the rating to between 0 and 1
            print(x)
            return tf.nn.sigmoid(x)
            #return movie_bias

model = RecommenderNet(num_users, num_movies, EMBEDDING_SIZE)
model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(), optimizer=keras.optimizers.Adam(lr=0.001)
    )
ratings = model.predict(that).flatten()

movies_not_watched.loc[:,'prediction'] = ratings
highest_score = ratings[ratings.argsort()[-10:]][::-1]

recom_movie_titles = movies_not_watched.loc[movies_not_watched.loc[:,'prediction'].isin(highest_score)]
recom_movie_titles
#movies_not_watched.loc[movies_not_watched['index'].isin(top_ratings_indices)]

/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


Number of users: 610, Number of Movies: 9721, Min rating: 0.5, Max rating: 5.0
Tensor("recommender_net_7/embedding_31/embedding_lookup/Identity:0", shape=(None, 1), dtype=float32)
Tensor("recommender_net_7/Tensordot:0", shape=(), dtype=float32)
Tensor("recommender_net_7/add_1:0", shape=(None, 1), dtype=float32)
Tensor("recommender_net_7/embedding_31/embedding_lookup/Identity:0", shape=(None, 1), dtype=float32)
Tensor("recommender_net_7/Tensordot:0", shape=(), dtype=float32)
Tensor("recommender_net_7/add_1:0", shape=(None, 1), dtype=float32)


,index,movieId,title,genres,fake_id,prediction
5825,5825,32179,Elevator to the Gallows (a.k.a. Frantic) (Asce...,Crime|Drama|Thriller,1,0.540940
5826,5826,32213,Cube Zero (2004),Horror|Mystery|Sci-Fi|Thriller,1,0.533786
5830,5830,32291,Melinda and Melinda (2004),Comedy|Drama,1,0.541096
5831,5831,32294,Milk and Honey (2003),Drama,1,0.540357
5832,5832,32296,Miss Congeniality 2: Armed and Fabulous (2005),Adventure|Comedy|Crime,1,0.535862
5840,5840,32440,If Looks Could Kill (1991),Action|Comedy,1,0.536827
5841,5841,32442,Greedy (1994),Comedy,1,0.537290
5842,5842,32456,"Pom Poko (a.k.a. Raccoon War, The) (Heisei tan...",Animation|Comedy|Drama|Fantasy,1,0.536170
5850,5850,32587,Sin City (2005),Action|Crime|Film-Noir|Mystery|Thriller,1,0.536366
5853,5853,32598,Fever Pitch (2005),Comedy|Romance,1,0.538157


In [12]:
recommender = RecommenderNet(num_users, num_movies, EMBEDDING_SIZE)
recommender

In [13]:
recommender.call(inputs)

NameError: name 'inputs' is not defined

In [8]:
tf.nn.sigmoid(x)

InvalidArgumentError: Value for attr 'T' of int64 is not in the list of allowed values: bfloat16, half, float, double, complex64, complex128
	; NodeDef: {{node Sigmoid}}; Op<name=Sigmoid; signature=x:T -> y:T; attr=T:type,allowed=[DT_BFLOAT16, DT_HALF, DT_FLOAT, DT_DOUBLE, DT_COMPLEX64, DT_COMPLEX128]> [Op:Sigmoid]

In [ ]:
There are two ways to instantiate a Model:

1 - With the "Functional API", where you start from Input, you chain layer calls to specify the model's forward pass, and finally you create your model from inputs and outputs:

import tensorflow as tf

inputs = tf.keras.Input(shape=(3,))
x = tf.keras.layers.Dense(4, activation=tf.nn.relu)(inputs)
outputs = tf.keras.layers.Dense(5, activation=tf.nn.softmax)(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

2 - By subclassing the Model class: in that case, you should define your layers in __init__ 
    and you should implement the model's forward pass in call.

import tensorflow as tf

class MyModel(tf.keras.Model):

  def __init__(self):
    super(MyModel, self).__init__()
    self.dense1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(5, activation=tf.nn.softmax)

  def call(self, inputs):
    x = self.dense1(inputs)
    return self.dense2(x)

model = MyModel()

If you subclass Model, you can optionally have a training argument (boolean) in call, which you can use to specify a different behavior in training and inference:

import tensorflow as tf

class MyModel(tf.keras.Model):

  def __init__(self):
    super(MyModel, self).__init__()
    self.dense1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(5, activation=tf.nn.softmax)
    self.dropout = tf.keras.layers.Dropout(0.5)

  def call(self, inputs, training=False):
    x = self.dense1(inputs)
    if training:
      x = self.dropout(x, training=training)
    return self.dense2(x)

model = MyModel()


In [ ]:
class Foo(Bar):
    def baz(self, arg):
        return super().baz(arg)

In [ ]:
class RecommenderNet(keras.Model):
        def __init__(self, num_users, num_movies, embedding_size, **kwargs):
            super(RecommenderNet, self).__init__(**kwargs)

In [ ]:
        def call(self, inputs):


In [ ]:

4

Actually __call__ method is implemented in the Layer class, which is inherited by Network class, 
which is inherited by Model class:

class Layer(module.Module):
    def __call__(self, inputs, *args, **kwargs):

class Network(base_layer.Layer):

class Model(network.Network):

So MyClass will inherit this __call__ method.

Additional info:

So actually what we do is overriding the inherited call method, which new call method will be then called 
from the inherited __call__ method. That is why we don't need to do a model.call(). 
So when we call our model instance, it's inherited __call__ method will be executed automatically, 
which calls our own call method.


In [14]:
model.summary()

Model: "recommender_net_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     multiple                  30500     
_________________________________________________________________
embedding_13 (Embedding)     multiple                  610       
_________________________________________________________________
embedding_14 (Embedding)     multiple                  487100    
_________________________________________________________________
embedding_15 (Embedding)     multiple                  9742      
Total params: 527,952
Trainable params: 527,952
Non-trainable params: 0
_________________________________________________________________


In [16]:
plot_model()

NameError: name 'plot_model' is not defined

In [ ]:
สิ่งที่มีอยู่ตอนนี้คือ flat_list ซึ่งคือ input genres shape = (7,7)
อยากได้ อะไรออกมา?

print(movie_bias) คือ Tensor("recommender_net_7/embedding_31/embedding_lookup/Identity:0", shape=(None, 1), dtype=float32)

embedding/embedding_lookup นี่ก็คือแค่ num_movies ซึ่งคือ input_dim ที่เราใส่ไว้เอง
แต่มันจะเอามาคำนวณอะไร
ถ้าสำหรับ movie_bias มันคือ ratings แล้วเราจะมี genres_bias ซึ่งคือค่า cosine_similarity
ของ genres แต่ละเรื่อง เมื่อเทียบกับ vectorized ของไอ้ 7 อันใน flat_list นั้นได้ป่าว
